<a href="https://colab.research.google.com/github/suresh-venkate/NLP_LLM/blob/main/Transformers/BERT/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Implementation - PyTorch



## Install Required Libraries

In [1]:
%%capture
pip install transformers datasets tokenizers

In [6]:
import os
from pathlib import Path
import re
import random
import tqdm
import itertools
import math
import numpy as np

import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

## Download dataset

Download cornell movie dialogs corpus ([Link](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html))

In [2]:
%%capture
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

## Load data into memory

In [3]:
### loading all data into memory
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

In [4]:
### split text
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [5]:
### generate question answer pairs
MAX_LEN = 64
pairs = []
ind = 0
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []

        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()
        second = lines_dic[ids[i+1]].strip()

        qa_pairs.append(' '.join(first.split()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)

## Wordpiece Tokenization

BERT employs a WordPiece tokenizer. To train the tokenizer, the BertWordPieceTokenizer from the transformer library is used with the steps below:



*   Saving the conversation text into multiple .txt files (with batch of N=10000)
*   Define BertWordPieceTokenizer with some parameters like **clean_text** to remove control characters, **handle_chinese_chars** to include spaces around Chinese characters, **stripe_accents** to remove accents and make é → e, ô → o, and **lowercase** to view capital and lowercase characters as equal.
*   Train the tokenizer based on the file path to .txt files with parameters like **vocab_size** defining the total number of tokens, **min_frequency** for minimum frequency for a pair of tokens to be merged, **special_tokens** defining a list of the special tokens that BERT uses, **limit_alphabet** for a maximum number of different characters, **workpieces_prefix** the prefix added to pieces of words (like ##ing).

In [14]:
### save data as txt file
os.mkdir('./data')
text_data = []
file_count = 0

In [15]:
for sample in tqdm.tqdm([x[0] for x in pairs]):
    text_data.append(sample)

    # once we hit the 10K mark, save to file
    if len(text_data) == 10000:
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 221616/221616 [00:00<00:00, 1228224.05it/s]


In [16]:
paths = [str(x) for x in Path('./data').glob('**/*.txt')]

The code below is used to train a custom tokenizer for a BERT model using the **BertWordPieceTokenizer** class. This tokenizer is designed to tokenize text data into subword tokens suitable for use with BERT-based models.

First, we define the tokenizer with the following parameters:

*   **clean_text**=True: This parameter specifies whether the text data should be cleaned before tokenization. When set to True, it performs text cleaning operations such as removing extra spaces and unwanted characters.
*   **handle_chinese_chars**=False: This parameter specifies whether the tokenizer should handle Chinese characters specially. If set to True, the tokenizer would treat Chinese characters as individual tokens; otherwise, it would treat them as parts of larger tokens.
*   **strip_accents**=False: This parameter specifies whether accents should be stripped from characters. When set to True, it removes accents from characters before tokenization
*   **lowercase**=True: This parameter specifies whether the text should be converted to lowercase before tokenization. When set to True, it converts all text to lowercase.

In [18]:
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

Then we train the tokenizer with the following parameters:

*   **files**=paths: This parameter specifies the list of file paths containing the text data used for training the tokenizer.
*   **vocab_size**=30_000: This parameter specifies the maximum size of the vocabulary (number of tokens) that the tokenizer will produce. In this case, the vocabulary size is set to 30,000 tokens.
* **min_frequency**=5: This parameter specifies the minimum frequency threshold for tokens to be included in the vocabulary. Tokens that occur less frequently than this threshold will not be included in the vocabulary.
* **limit_alphabet**=1000: This parameter specifies the maximum number of characters in the alphabet used for tokenization. It limits the characters that the tokenizer considers when building the vocabulary.
* **wordpieces_prefix**='##': This parameter specifies the prefix to be used for wordpieces. Wordpieces are subword tokens produced by the tokenizer. The ## prefix indicates that a token is a continuation of a previous token.
* **special_tokens**=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']: This parameter specifies a list of special tokens to be included in the vocabulary. These tokens have specific meanings within the BERT model architecture, such as padding token [PAD], classification token [CLS], separator token [SEP], mask token [MASK], and unknown token [UNK].

In [20]:
tokenizer.train(
    files=paths,
    vocab_size=30_000,
    min_frequency=5,
    limit_alphabet=1000,
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )